# Cas Donoghue
##  Database Analyst Technical Assessment
## 01NOV2017

#### Disclaimer: I used internet resources for reference in solving these problems. I DID NOT discuss questions with any colleagues. 

## Feel free to toggle on/off some of the underlying python code, HOWEVER it should not be necessary to understand my solutions

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle on/off the underlying python code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
import sqlite3 as lite # for database
import pandas as pd # good ol pandas
import warnings # catch a warning that makes my HTML look bad
import os # the joy of context...

# point to data
DATA_DIR = os.path.join(os.getcwd(), 'source_csv')

In [3]:
def df_dict_to_lite_db(database_name, dataframe_dict):
    """ Create a SQLite3 DB with tables named as key in dictionary having content of value dataframe
        ARGS: Name of the SQLITE database output file (will suffix with '.db'), dictionary of dataframes where
        the key is the desired table name, value is the dataframe representing data to be inserted into table
        RETURN: absolute path to database
    """
    
    # Make single file DB for part_b
    append_db = database_name + '.db'
    full_path = os.path.join(os.getcwd(), append_db)
    conn = lite.connect(full_path)

    # Declare and fill tables with parsed data
    # note there is a UserWarning about the space in the column name 'Hire Date'.
    # I choose to catch that and not let it distract visual presentation
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        for table_name, data_frame in dataframe_dict.items():
            data_frame.to_sql(table_name, conn, if_exists='replace', index=False)
            
    conn.close()
    return full_path


def execute_statement(query, database):
    """ Execute a query and return results as dataframe. 
        ARGS: SQL statement, database path
        RETURN: dataframe with results
    """
    conn = lite.connect(database)
    result = pd.read_sql_query(query, conn)
    conn.close()
    return result

# Part A 

### Disclaimer, I tried to avoid just looking up definitions and parroting them back. For this section I tended to just write initial thoughts.

1) What is a join? <br>
    A join is the glue used to connect tables. In this analogy instead of polymer connecting solids it is relational algebra satisfying conditions. To further the analogy you use different polymers for different applications and when you have two very large surfaces to join the polymerization reaction takes more time. 
    
2) What are the types of join? Explain each (2 sentences or less) <br>

 - INNER: rows that satisfy condition of key in both tables<br>
 - RIGHT: all rows from right, rows that satsify condition from left else null<br>
 - LEFT: do a mental find-replace 'right'->'left' in RIGHT definition.<br>
 - OUTER: rows from both left and right<br>
 - CROSS: cartesian product, every combination of rows (have never used this, big table would result...)<br>
 
3) What are the differences between a primary key, a unique key, and a foreign key? <br>
   - Primary key is unique row identifier, must be one and only one per table and it cannot be Null.
   - Unique key is very much like a primary key in that it must be a unique value in the feild of the table it represents (can have one Null instance as well). It is like an under-privilaged primary key.
   - A foreign key is a key in a table that represents a primary/unique key in another table. Foreign keys can have duplicate values or be null. 


4) What are the different standard normalizations?  Give an example of each. <br>
 - 1NF: primary key, one value indexed by (row,col)
 - 2NF: be 1NF and enforce separations of tables, IE do not repeat keys over and over
 - 3NF: organize tables such that they only depend on a certain key, this helps reduce data redundancy in a specific table
 "Every non-key attribute must provide a fact about the key, the whole key, and nothing but the key. So help me Codd"

### 1NF
- Primay key is id
- No duplicate values 

In [4]:
pd.DataFrame(dict(name=['Andrew', 'Albert', 'Alex'], id=[11, 12, 13]))

,id,name
0,11,Andrew
1,12,Albert
2,13,Alex


### 2NF
- use example of 1NF in context below (which is also 2NF) in context of below, classes would have a table like students table mapping an id to class name. 

In [5]:
pd.DataFrame(dict(class_name=['French', 'French', 'Math'], student_id=[11, 12, 13]))

,class_name,student_id
0,French,11
1,French,12
2,Math,13


### 3NF
- satisfies 1NF and 2NF
- make sure no tranitive dependencies exist. 
- basically store many to many keys to provide mappings without making row data depended on that instance
- consider following frame in imaginary system modeling the 1NF and 2NF examples above

In [6]:
pd.DataFrame(dict(teach_id=[1,2,3], class_id=[11, 12, 13]))

,class_id,teach_id
0,11,1
1,12,2
2,13,3


5) What is a Database Relationship, and what are three of the more common ones? <br>
 - A relationship can be described in terms of cardinality. This is meant to describe how many expected values a key should be expected to join. 
 - One-to-one: each unique key being joined will have one entry in corresponding table (not very helpful). Ex: Student to student ID
 - One-to-many: Each key could match to multiple places. Ex: Student to places lived
 - Many-to-many: Multiple mappings on both sides. Ex: Student table could have a feild classes classes can have a field studends. 
 
6) What is a NULL value and how does it differ from a zero value?<br>
 - The NULL value is mean to represent truly nothing where zero represents the numerical value 0. NULLs are quite useful when doing aggregation functions as they are ignored. Comparisons with NULL must be carefully considered. 
    
7) Where can you get more information about BigQuery Standard SQL functions? <br>
 - Sooo many places on the internet!! 
 - official docs https://cloud.google.com/bigquery/docs/reference/standard-sql/
 - stack overflow https://stackoverflow.com/questions/tagged/google-bigquery
 - youtube, google cloud platform on Slack. 

# Part B

### Given the following data solve questions 1-4

1. How much did each Salesperson sell in May 2017?
2. On average, how long does it take to close an order?
3. For each order, how long was the Salesperson at the company when the deal was closed? 
4. List all customers who closed an order in May 2017, from the highest Amount to the lowest.


In [7]:
# Read in data to dictionary of dataframes(saved in CSV format)
part_b_source = dict()
part_b_source['Salesperson'] = pd.read_csv(os.path.join(DATA_DIR, 'Salesperson_B.csv'), parse_dates=['Hire Date'])
part_b_source['Customer'] = pd.read_csv(os.path.join(DATA_DIR, 'Customer_B.csv'))
part_b_source['Order'] = pd.read_csv(os.path.join(DATA_DIR, 'Order_B.csv'), parse_dates=['CreatedDate', 'ClosedDate'])

## Salesperson

In [8]:
part_b_source['Salesperson']

,ID,Name,Hire Date
0,1,Nancy,2006-04-01
1,2,Chuck,2014-03-27
2,3,Lora,2010-01-19
3,4,Phoebe,2011-01-19
4,5,Clarence,2017-05-01


## Customer

In [9]:
part_b_source['Customer']

,ID,Name,City
0,1,Green,Portland
1,2,Sapphire,San Francisco
2,3,Red,Tulsa
3,4,Amber,Portland
4,5,Neon,Columbus


## Order
### # will note here that table name as SQL keyword is unfortunate...

In [10]:
part_b_source['Order']

,ID,CreatedDate,ClosedDate,CustID,SalesID,Amount
0,1,2015-04-14,2017-05-30,4,3,100000
1,2,2017-04-01,2017-05-30,4,3,1000
2,3,2016-03-27,2017-05-01,1,1,3250
3,4,2016-02-26,2017-05-14,1,1,2000
4,5,2017-04-30,2017-05-06,5,1,12000
5,6,2017-04-15,2017-05-22,2,4,5000
6,7,2017-05-02,2017-05-03,3,5,550


# Make a SQLite database
- For the purpose of these exersises a single file database that supports ANSI standard SQL seems sufficient
- I will note that I have most experience using Oracle RDMS


In [11]:
part_b_db = df_dict_to_lite_db('part_b', part_b_source)

### Execute following query to show how I interact with Sqlite DB
```SQL
SELECT * FROM 'Order' o
INNER JOIN Customer c
ON o.CustID = c.ID
INNER JOIN Salesperson s
ON o.SalesID = s.id```

In [12]:
# This is the pattern I will use for database interaction
# I write the query as a string and return results as a dataframe
# note that this is "vanilla" SQL, no pandas tricks

query = '''
SELECT * FROM 'Order' o
INNER JOIN Customer c
ON o.CustID = c.ID
INNER JOIN Salesperson s
ON o.SalesID = s.id'''

execute_statement(query, part_b_db)

,ID,CreatedDate,ClosedDate,CustID,SalesID,Amount,ID,Name,City,ID,Name,Hire Date
0,1,2015-04-14 00:00:00,2017-05-30 00:00:00,4,3,100000,4,Amber,Portland,3,Lora,2010-01-19 00:00:00
1,2,2017-04-01 00:00:00,2017-05-30 00:00:00,4,3,1000,4,Amber,Portland,3,Lora,2010-01-19 00:00:00
2,3,2016-03-27 00:00:00,2017-05-01 00:00:00,1,1,3250,1,Green,Portland,1,Nancy,2006-04-01 00:00:00
3,4,2016-02-26 00:00:00,2017-05-14 00:00:00,1,1,2000,1,Green,Portland,1,Nancy,2006-04-01 00:00:00
4,5,2017-04-30 00:00:00,2017-05-06 00:00:00,5,1,12000,5,Neon,Columbus,1,Nancy,2006-04-01 00:00:00
5,6,2017-04-15 00:00:00,2017-05-22 00:00:00,2,4,5000,2,Sapphire,San Francisco,4,Phoebe,2011-01-19 00:00:00
6,7,2017-05-02 00:00:00,2017-05-03 00:00:00,3,5,550,3,Red,Tulsa,5,Clarence,2017-05-01 00:00:00


# 1. How much did each Salesperson sell in May 2017?
## Assumptions
- The sale is not final until it is closed. Therefore for this metric we will count sales that have closed in May2017 as total sales. 
- Every value in Order.SalesID field maps to a corresponding value in Salesperson.ID 

## notes 
- You will notice that I tend to default to using the WITH syntax over writing sub queries
- The general idea is to narrow down the set of data to observe (presumably 'Order' has an been indexed on ClosedDate)
- Then the data is aggregated on unique sales IDs (using both the sum and count method) 
- Once the "hard work" has been done (filter and aggregation) join in the names. This prevents unnecessary join work
- I chose to report on both total sale volume as a sum of revenue as well as total number of sales

## Solution

```SQL
WITH may_sales AS (
SELECT SalesID, SUM(Amount) AS sum_sales, COUNT(Amount) AS count_sales
FROM 'Order' 
WHERE ClosedDate BETWEEN date('2017-05-01') AND date('2017-05-31')
GROUP BY SalesID
)
SELECT s.Name, ms.sum_sales AS 'Total Revenue', ms.count_sales AS 'Number of Sales'  
FROM may_sales ms
INNER JOIN Salesperson s
ON ms.SalesID = s.ID
ORDER BY ms.sum_sales DESC

```

In [13]:
query = '''
WITH may_sales AS (
SELECT SalesID, SUM(Amount) AS sum_sales, COUNT(Amount) AS count_sales
FROM 'Order' 
WHERE ClosedDate BETWEEN date('2017-05-01') AND date('2017-05-31')
GROUP BY SalesID
)
SELECT s.Name, ms.sum_sales AS 'Total Revenue', ms.count_sales AS 'Number of Sales'  
FROM may_sales ms
INNER JOIN Salesperson s
ON ms.SalesID = s.ID
ORDER BY ms.sum_sales DESC
'''
execute_statement(query, part_b_db)

,Name,Total Revenue,Number of Sales
0,Lora,101000,2
1,Nancy,17250,3
2,Phoebe,5000,1
3,Clarence,550,1


# 2. On average, how long does it take to close an order?
## Notes
- This seems to be a total "got-cha" problem?? I am assuming that there will always be an open and closed date and that closed is always "greater" than open. 
- Maybe it is kind of cheating that the SQLite function takes care of date time deltas and simply returns days?
- When I do this kind of thing at Oracle, whe have custom functions that essentially map datetimes to seconds and then perform the conversions to days/weeks/quaters as needed. 

## Solution
```SQL
SELECT AVG(julianday(o.ClosedDate) - julianday(o.CreatedDate)) AS 'Average Days to Close'
FROM 'Order' o
```

In [14]:
query= '''SELECT AVG(julianday(o.ClosedDate) - julianday(o.CreatedDate)) AS 'Average Days to Close'
FROM 'Order' o'''

execute_statement(query, part_b_db)

,Average Days to Close
0,246.142857


# 3. For each order, how long was the Salesperson at the company when the deal was closed? 
## Assumptions
- Assuming we want to identify the order id, the salesperson name and the number of days they have been with the company when they closed each deal

## Notes
- For the purpose of these simple example I have been using spaces in column names. This is not something I ever do in practice. I favor underscores for separation over camel case or other methods. I bring this up because 'Hire Date" was a column value in the Salesperson table. If this were not a toy example I would push for changing that column name. 

## Solution 

```SQL
SELECT o.ID AS 'Order ID',
    s.Name AS 'Salesperson Name', 
    julianday(o.ClosedDate) - julianday(s.'Hire Date') AS 'Days Worked Between Hire and Closing'
FROM 'Order' o
INNER JOIN Salesperson s
ON o.SalesID = s.ID
```

In [15]:
query = '''
SELECT o.ID AS 'Order ID',
    s.Name AS 'Salesperson Name', 
    julianday(o.ClosedDate) - julianday(s.'Hire Date') AS 'Days Worked Between Hire and Closing'
FROM 'Order' o
INNER JOIN Salesperson s
ON o.SalesID = s.ID
'''

execute_statement(query, part_b_db)

,Order ID,Salesperson Name,Days Worked Between Hire and Closing
0,1,Lora,2688.0
1,2,Lora,2688.0
2,3,Nancy,4048.0
3,4,Nancy,4061.0
4,5,Nancy,4053.0
5,6,Phoebe,2315.0
6,7,Clarence,2.0


# 4. List all customers who closed an order in May 2017, from the highest Amount to the lowest.
## Notes
- Again I separate concernes using the With syntax. It is probably overkill for this example as a simple where condition would do the same thing. I stick with it though because when possible I fundamentally like to filter and aggregate tables and then perform joins explicitly later to gather necessary information. 
- The first solution shows the dates and the indivdual purchase information by customer. I suppose that we may be interested in considering customers in terms of the total amount they purchased in May 2017. For this reason I also include a query that aggregates on customer and orders by the sum of total purchases. 
- I also take the opportunity to NOT use the WITH syntax for the aggregate solution As it really is overkill...

## Solution (using WITH syntax)
```SQL
WITH may_sales AS (
SELECT CustID, Amount, ClosedDate
FROM 'Order' 
WHERE ClosedDate BETWEEN date('2017-05-01') AND date('2017-05-31')
)
SELECT c.Name AS Customer, ms.ClosedDate AS 'Closing Date', ms.Amount AS 'Sale Amount'
FROM may_sales ms
INNER JOIN Customer c
ON ms.CustID = c.ID
ORDER BY ms.Amount DESC
```

## Group by customer agregate sum of total spent

```SQL
SELECT c.Name AS Customer, SUM(o.Amount) AS 'Sale Amount'
FROM 'Order' o
INNER JOIN Customer c
ON o.CustID = c.ID
WHERE o.ClosedDate BETWEEN date('2017-05-01') AND date('2017-05-31')
GROUP BY c.Name
ORDER BY SUM(o.Amount) DESC
```


## Solution (using WITH syntax)

In [16]:
# with syntax
query = '''
WITH may_sales AS (
SELECT CustID, Amount, ClosedDate
FROM 'Order' 
WHERE ClosedDate BETWEEN date('2017-05-01') AND date('2017-05-31')
)
SELECT c.Name AS Customer, ms.ClosedDate AS 'Closing Date', ms.Amount AS 'Sale Amount'
FROM may_sales ms
INNER JOIN Customer c
ON ms.CustID = c.ID
ORDER BY ms.Amount DESC
'''

execute_statement(query, part_b_db)

,Customer,Closing Date,Sale Amount
0,Amber,2017-05-30 00:00:00,100000
1,Neon,2017-05-06 00:00:00,12000
2,Sapphire,2017-05-22 00:00:00,5000
3,Green,2017-05-01 00:00:00,3250
4,Green,2017-05-14 00:00:00,2000
5,Amber,2017-05-30 00:00:00,1000
6,Red,2017-05-03 00:00:00,550


## Group by customer agregate sum of total spent

In [17]:
## Aggregate
query = '''
SELECT c.Name AS Customer, SUM(o.Amount) AS 'Sale Amount'
FROM 'Order' o
INNER JOIN Customer c
ON o.CustID = c.ID
WHERE o.ClosedDate BETWEEN date('2017-05-01') AND date('2017-05-31')
GROUP BY c.Name
ORDER BY SUM(o.Amount) DESC
'''

execute_statement(query, part_b_db)

,Customer,Sale Amount
0,Amber,101000
1,Neon,12000
2,Green,5250
3,Sapphire,5000
4,Red,550


# Part C

Given the tables below, write a query that will provide all of the available customer information for customers that have purchased at least 25 apples

In [18]:
# Read in data to dictionary of dataframes(saved in CSV format)
part_c_source = dict()
part_c_source['Customers'] = pd.read_csv(os.path.join(DATA_DIR, 'Customers_C.csv'), parse_dates=['CustomerSince'])
part_c_source['Orders'] = pd.read_csv(os.path.join(DATA_DIR, 'Orders_C.csv'), parse_dates=['DateShipped'])
part_c_source['Products'] = pd.read_csv(os.path.join(DATA_DIR, 'Products_C.csv'))
part_c_source['Order_Details'] = pd.read_csv(os.path.join(DATA_DIR, 'Order_Details_C.csv'))

# Customers

In [19]:
part_c_source['Customers']

,ID,Name,City,CustomerSince,Industry,NumEmployees
0,1,Rose,Nashville,2014-04-01,Healthcare,50-100
1,2,Pearl,Seattle,2013-03-27,Healthcare,"1,000+"
2,3,Jabberwocky,Portland,2016-08-01,Technology,"500-1,000"
3,4,Jensen,San Diego,2015-04-01,Finance,"1,000+"


# Orders

In [20]:
part_c_source['Orders']

,ID,CustomerID,DateShipped
0,1,1,2017-05-17
1,2,2,2017-05-19
2,3,1,2017-05-16
3,4,2,2017-05-20
4,5,3,2017-05-14


# Products

In [21]:
part_c_source['Products']

,ID,Name
0,1,Apple
1,2,Bananas
2,3,Cockatoos


# Order_Details

In [22]:
part_c_source['Order_Details']

,OrderID,ProductID,Quantity
0,1,1,50
1,1,2,2
2,1,3,7
3,2,2,10
4,3,2,10
5,3,1,100
6,4,3,2
7,5,1,10
8,5,2,100
9,5,3,2


# Make single file DB for part_c

In [23]:
part_c_db = df_dict_to_lite_db('part_c', part_c_source)

#  Provide all of the available customer information for customers that have purchased at least 25 apples

## Assumptions
- This is a pretty open ended question. I interpret the intent to be: filter on every unique customer who has accumulated a total of 25 apples or more in all of their order history. To this end I group by customer and aggregate the sum of apple quantities.
- In terms of "all available customer information" this could be simply the rows in "Customers" describing customers who have accumulated at least 25 apple purchases. This is kind of a boring result though, I think what would be more interesting is the entire purchase history for every customer who has accumulated over 25 apples. This data could be used to understand customer habits, to make predictions on their future purchases or even to make suggestions for other products we would ike to sell them. 

## Notes
- total_apple_by_cust_ID gets the sum of apples by customerID. This allows us to understand which customers have accumulated at least 25 apple purchases from us. Note how the WHERE condition is set using the IN syntax as the result of a sub query. This would handle the case where multiple things called "Apple" have disparate product codes. It also prevents "hard coding" of keys. 
- cust_over_25 simply gathers all customer ID's who have ever accumulated 25 apple purchases
- customer table is filtered on customer ID's in cust_over_25

## Customer Table Only Solution

```SQL
WITH total_apple_by_cust_ID AS (
SELECT od.ProductID,
    o.CustomerID,
    SUM(od.Quantity) AS aggregated_quantity
FROM Order_Details od
INNER JOIN Orders o 
ON od.OrderID = o.ID
WHERE ProductID IN (SELECT ID FROM Products WHERE Name = 'Apple')
GROUP BY od.ProductID, o.CustomerID),

cust_over_25 AS (
SELECT CustomerID 
FROM total_apple_by_cust_ID
WHERE aggregated_quantity >= 25
)

SELECT * FROM Customers c
WHERE c.ID IN (SELECT CustomerID FROM cust_over_25)
```

## Customer Order History Solution

### As noted above this solution finds the entire order history for customers who have every bought at least 25 apples

```SQL
WITH total_apple_by_cust_ID AS (
SELECT od.ProductID,
    o.CustomerID,
    SUM(od.Quantity) AS aggregated_quantity
FROM Order_Details od
INNER JOIN Orders o 
ON od.OrderID = o.ID
WHERE ProductID IN (SELECT ID FROM Products WHERE Name = 'Apple')
GROUP BY od.ProductID, o.CustomerID),

cust_over_25 AS (
SELECT CustomerID 
FROM total_apple_by_cust_ID
WHERE aggregated_quantity >= 25
), 

order_history_by_cust AS (
SELECT c.Name AS 'Customer Name',
    c.City AS 'City',
    c.CustomerSince AS 'Customer Since',
    c.Industry AS 'Industry',
    c.NumEmployees AS 'Num Employees',
    od.OrderID AS 'Order ID',
    p.Name AS 'Product Name',
    od.Quantity AS 'Num Purchased',
    o.DateShipped AS 'Date Shipped'
FROM Order_Details od
INNER JOIN Orders o
ON od.OrderID = o.ID
INNER JOIN Products p
ON od.ProductID = p.ID
INNER JOIN Customers c
ON o.CustomerID = c.ID
WHERE o.CustomerID IN (SELECT CustomerID FROM cust_over_25)
)

SELECT * FROM order_history_by_cust
-- SELECT * FROM total_apple_by_cust_ID
-- SELECT * FROMcust_over_25
```

#### Final note: This example highlights why I make such heavy use of the WITH syntax. The commented out temporary tables will allow me to understand what is happening at intermediate views. There is nothing worse than trying to extract a subquery nested in 10 SELECT FROM ()s just to debug or improve...


## Customer Table Only Solution

In [24]:
query='''
WITH total_apple_by_cust_ID AS (
SELECT od.ProductID,
    o.CustomerID,
    SUM(od.Quantity) AS aggregated_quantity
FROM Order_Details od
INNER JOIN Orders o 
ON od.OrderID = o.ID
WHERE ProductID IN (SELECT ID FROM Products WHERE Name = 'Apple')
GROUP BY od.ProductID, o.CustomerID),

cust_over_25 AS (
SELECT CustomerID 
FROM total_apple_by_cust_ID
WHERE aggregated_quantity >= 25
)

SELECT * FROM Customers c
WHERE c.ID IN (SELECT CustomerID FROM cust_over_25)
'''
execute_statement(query, part_c_db)

,ID,Name,City,CustomerSince,Industry,NumEmployees
0,1,Rose,Nashville,2014-04-01 00:00:00,Healthcare,50-100


## Customer Order History Solution

In [25]:
query='''
WITH total_apple_by_cust_ID AS (
SELECT od.ProductID,
    o.CustomerID,
    SUM(od.Quantity) AS aggregated_quantity
FROM Order_Details od
INNER JOIN Orders o 
ON od.OrderID = o.ID
WHERE ProductID IN (SELECT ID FROM Products WHERE Name = 'Apple')
GROUP BY od.ProductID, o.CustomerID),

cust_over_25 AS (
SELECT CustomerID 
FROM total_apple_by_cust_ID
WHERE aggregated_quantity >= 25
), 

order_history_by_cust AS (
SELECT c.Name AS 'Customer Name',
    c.City AS 'City',
    c.CustomerSince AS 'Customer Since',
    c.Industry AS 'Industry',
    c.NumEmployees AS 'Num Employees',
    od.OrderID AS 'Order ID',
    p.Name AS 'Product Name',
    od.Quantity AS 'Num Purchased',
    o.DateShipped AS 'Date Shipped'
FROM Order_Details od
INNER JOIN Orders o
ON od.OrderID = o.ID
INNER JOIN Products p
ON od.ProductID = p.ID
INNER JOIN Customers c
ON o.CustomerID = c.ID
WHERE o.CustomerID IN (SELECT CustomerID FROM cust_over_25)
)

SELECT * FROM order_history_by_cust
-- SELECT * FROM total_apple_by_cust_ID
-- SELECT * FROMcust_over_25
 
'''
execute_statement(query, part_c_db)

,Customer Name,City,Customer Since,Industry,Num Employees,Order ID,Product Name,Num Purchased,Date Shipped
0,Rose,Nashville,2014-04-01 00:00:00,Healthcare,50-100,1,Apple,50,2017-05-17 00:00:00
1,Rose,Nashville,2014-04-01 00:00:00,Healthcare,50-100,1,Bananas,2,2017-05-17 00:00:00
2,Rose,Nashville,2014-04-01 00:00:00,Healthcare,50-100,1,Cockatoos,7,2017-05-17 00:00:00
3,Rose,Nashville,2014-04-01 00:00:00,Healthcare,50-100,3,Apple,100,2017-05-16 00:00:00
4,Rose,Nashville,2014-04-01 00:00:00,Healthcare,50-100,3,Bananas,10,2017-05-16 00:00:00


# Part D 

```SQL
SELECT
 ticket_id,
 MAX(operating_system) operating_system,
 MAX(difficulty) difficulty,
 MAX(environment) environment,
 MAX(escalation) escalation,
 MAX(nature_of_issue) nature_of_issue,
 MAX(needs_assistance) needs_assistance,
 MAX(organization) organization,
 MAX(support_level) support_level,
FROM (
 SELECT
   ticket_id,
   (CASE
       WHEN title = 'Operating System' THEN value
       ELSE NULL END) AS operating_system,
   (CASE
       WHEN title = 'Difficulty' THEN value
       ELSE NULL END) AS difficulty,
   (CASE
       WHEN title = 'Environment' THEN value
       ELSE NULL END) AS environment,
   (CASE
       WHEN title = 'Escalation' THEN value
       ELSE NULL END) AS escalation,
   (CASE
       WHEN title = 'Nature of Issue' THEN value
       ELSE NULL END) AS nature_of_issue,
   (CASE
       WHEN title = 'Needs Assistance' THEN value
       ELSE NULL END) AS needs_assistance,
   (CASE
       WHEN title = 'Organization' THEN value
       ELSE NULL END) AS organization,
   (CASE
       WHEN title = 'Priority' THEN value
       ELSE NULL END) AS priority,
   (CASE
       WHEN title = 'Support Level' THEN value
       ELSE NULL END) AS support_level
 FROM
   zendesk.tickets ) AS temp
GROUP BY
 Ticket_id ```
 
 
1. In layman’s terms, what is the above query doing? 
2. Given the information available in the query, what does zendesk.tickets and the new table that results from this query look like? Describe the schema or diagram the tables below. 
3. What, if anything, is odd about this query?


In [26]:
part_d_src = pd.DataFrame(dict(ticket_id=[1,2,2,4,5,7,7,8,9,10],
                               title=['Operating System', 'Difficulty',
                                      'Environment', 'Escalation', 'Nature of Issue',
                                      'Needs Assistance', 'Needs Assistance', 'Organization',
                                      'Priority', 'Support Level']))

In [27]:
part_d_db = df_dict_to_lite_db('part_d', dict(tickets=part_d_src))

# 1. In layman’s terms, what is the above query doing? 
- First off, line 10 appears to have an extra comma ( MAX(support_level) support_level,)
- Second i dont think the parenthesis ending the select statement followed by the alias temp is quite right. 
- However the purpose of a query like this is bascially a pivot table. I will show by example. 
- In laymans terms we want to take essentially two columns and expand them to 10 colums where data for a specific ticket is descrived on one row instead of a having multiple entries in a column. 

# Imagine that zendesk.tickets has these two columns
- there is a ticket id and a title for portion of the ticket. 

In [28]:
part_d_src

,ticket_id,title
0,1,Operating System
1,2,Difficulty
2,2,Environment
3,4,Escalation
4,5,Nature of Issue
5,7,Needs Assistance
6,7,Needs Assistance
7,8,Organization
8,9,Priority
9,10,Support Level


# Pivot 
- We want to essentially group by ticket id and place a value in a column for each of the different lables (or NULL if there is no label). 
- I run the query on example data to show how for example ticket id 2 which has 2 lables (titles) will have one row withthe value in the corresponding columns. 

```SQL
SELECT
 ticket_id,
 MAX(operating_system) operating_system,
 MAX(difficulty) difficulty,
 MAX(environment) environment,
 MAX(escalation) escalation,
 MAX(nature_of_issue) nature_of_issue,
 MAX(needs_assistance) needs_assistance,
 MAX(organization) organization,
 MAX(support_level) support_level
FROM (
 SELECT
   ticket_id,
   (CASE
       WHEN title = 'Operating System' THEN '__value__'
       ELSE NULL END) AS operating_system,
   (CASE
       WHEN title = 'Difficulty' THEN '__value__'
       ELSE NULL END) AS difficulty,
   (CASE
       WHEN title = 'Environment' THEN '__value__'
       ELSE NULL END) AS environment,
   (CASE
       WHEN title = 'Escalation' THEN '__value__'
       ELSE NULL END) AS escalation,
   (CASE
       WHEN title = 'Nature of Issue' THEN '__value__'
       ELSE NULL END) AS nature_of_issue,
   (CASE
       WHEN title = 'Needs Assistance' THEN '__value__'
       ELSE NULL END) AS needs_assistance,
   (CASE
       WHEN title = 'Organization' THEN '__value__'
       ELSE NULL END) AS organization,
   (CASE
       WHEN title = 'Priority' THEN '__value__'
       ELSE NULL END) AS priority,
   (CASE
       WHEN title = 'Support Level' THEN '__value__'
       ELSE NULL END) AS support_level
 FROM
   tickets )
GROUP BY
 Ticket_id
```

# output description
- As you can see the values were pivoted and to obtain the structure below. Note ticket_id 2 to see how multiple entries row wise condense to a single row. 

In [29]:
query = '''
SELECT
 ticket_id,
 MAX(operating_system) operating_system,
 MAX(difficulty) difficulty,
 MAX(environment) environment,
 MAX(escalation) escalation,
 MAX(nature_of_issue) nature_of_issue,
 MAX(needs_assistance) needs_assistance,
 MAX(organization) organization,
 MAX(support_level) support_level
FROM (
 SELECT
   ticket_id,
   (CASE
       WHEN title = 'Operating System' THEN '__value__'
       ELSE NULL END) AS operating_system,
   (CASE
       WHEN title = 'Difficulty' THEN '__value__'
       ELSE NULL END) AS difficulty,
   (CASE
       WHEN title = 'Environment' THEN '__value__'
       ELSE NULL END) AS environment,
   (CASE
       WHEN title = 'Escalation' THEN '__value__'
       ELSE NULL END) AS escalation,
   (CASE
       WHEN title = 'Nature of Issue' THEN '__value__'
       ELSE NULL END) AS nature_of_issue,
   (CASE
       WHEN title = 'Needs Assistance' THEN '__value__'
       ELSE NULL END) AS needs_assistance,
   (CASE
       WHEN title = 'Organization' THEN '__value__'
       ELSE NULL END) AS organization,
   (CASE
       WHEN title = 'Priority' THEN '__value__'
       ELSE NULL END) AS priority,
   (CASE
       WHEN title = 'Support Level' THEN '__value__'
       ELSE NULL END) AS support_level
 FROM
   tickets )
GROUP BY
 Ticket_id
'''
execute_statement(query, part_d_db)

,ticket_id,operating_system,difficulty,environment,escalation,nature_of_issue,needs_assistance,organization,support_level
0,1,__value__,None,None,None,None,None,None,None
1,2,None,__value__,__value__,None,None,None,None,None
2,4,None,None,None,__value__,None,None,None,None
3,5,None,None,None,None,__value__,None,None,None
4,7,None,None,None,None,None,__value__,None,None
5,8,None,None,None,None,None,None,__value__,None
6,9,None,None,None,None,None,None,None,None
7,10,None,None,None,None,None,None,None,__value__


# 3. What, if anything, is odd about this query?
- There is no definition for the value variable in the case statements. For the example as you see above i just assinged the string '\__value__' in place of the missing information. 
- Also the (in my experience maing this toy example) typos regarding the extra comma in line 10 and the somewhat elusive problem of the closing parens before aliasing the query as temp. NOTE: these are likely due to my somewhat hacky way of illustrating this example, i'm sure in larger context this query will be fine. 